In [1]:
import glob
import pandas as pd
import numpy as np
import os
from IPython.display import HTML
import scipy

In [2]:
!ls ../../ppc/scripts

autotune_12000.sh
autotune_eval_10000.sh
autotune_eval_12000_seed.sh
autotune_eval_12000.sh
autotune_eval_8000.sh
autotune_eval_zifa_A.sh
autotune_eval_zifa_B.sh
autotune_eval_zifa_C_new.sh
autotune_eval_zifa_C.sh
autotune_eval_zifa_D_new.sh
autotune_eval_zifa_D.sh
autotune_eval_zifa_F_new.sh
autotune_eval_zifa_F.sh
autotune_eval_zifa_F_sos10.sh
autotune_eval_zifa_G_new.sh
autotune_eval_zifa_G.sh
autotune_eval_zifa_H.sh
autotune_eval_zifa_I.sh
autotune.sh
autotune_zi_12000.sh
autotune_zifa_new.sh
autotune_zifa.sh
autotune_zi.sh
aws_sos
brain_small
brain_small_old
brain_small.zip
catchup_new.txt
catchup.txt
check_hemato.py
cortex
cortex_bis
cortex_old
cortex.zip
data
hemato
hemato_old
hemato_zinb_1200_50_results.json
hemato_zinb_1200_50_results_new.json
hemato.zip
klein_dataset
klein_dataset_nb_1200_50_results.json
klein_dataset_zinb_1200_50_results.json
log_poisson_nb_dataset_10000
log_poisson_nb_dataset_10000_old1
log_poisson_nb_dataset_10000_sim1
log_poisson_nb_dataset_12000
log_pois

In [3]:


data_path = '../../ppc/scripts'
dataset_names = {'log_poisson_zifa_dataset_12000_0.08_{}'.format(zifa_lambda):
                 '$\lambda = {}$'.format(str(zifa_lambda).rstrip('0').rstrip('.'))
                 for zifa_lambda in [0., 1e-3, 1e-2, 1e-1, 1.0, 10.0]}
data_files = {}
data_names = {}
for dataset_name in dataset_names:
    data_files[dataset_name] = sorted(glob.glob(os.path.join(data_path, '{}/*.p'.format(dataset_name))))
    data_names[dataset_name] = []
    for data_file in data_files[dataset_name]:
        data_name = data_file.split('/')[-1].split('_')[0]
        data_names[dataset_name].append(data_name)

for dataset_name in list(dataset_names.keys()):
    if len(data_names[dataset_name]) != 2:
        del dataset_names[dataset_name]
        del data_files[dataset_name]
        del data_names[dataset_name]

In [4]:
dataset_names

{'log_poisson_zifa_dataset_12000_0.08_0.0': '$\\lambda = 0$',
 'log_poisson_zifa_dataset_12000_0.08_0.001': '$\\lambda = 0.001$',
 'log_poisson_zifa_dataset_12000_0.08_0.01': '$\\lambda = 0.01$',
 'log_poisson_zifa_dataset_12000_0.08_0.1': '$\\lambda = 0.1$',
 'log_poisson_zifa_dataset_12000_0.08_1.0': '$\\lambda = 1$',
 'log_poisson_zifa_dataset_12000_0.08_10.0': '$\\lambda = 10$'}

In [5]:
data_names

{'log_poisson_zifa_dataset_12000_0.08_0.0': ['nb', 'zinb'],
 'log_poisson_zifa_dataset_12000_0.08_0.001': ['nb', 'zinb'],
 'log_poisson_zifa_dataset_12000_0.08_0.01': ['nb', 'zinb'],
 'log_poisson_zifa_dataset_12000_0.08_0.1': ['nb', 'zinb'],
 'log_poisson_zifa_dataset_12000_0.08_1.0': ['nb', 'zinb'],
 'log_poisson_zifa_dataset_12000_0.08_10.0': ['nb', 'zinb']}

In [6]:
df = {}
for dataset_name in dataset_names:
    dfs = []
    for data_name, f in zip(data_names[dataset_name], data_files[dataset_name]):
        if data_name != 'zifa_full':

            #my_df = pd.read_csv(f, sep='\t')
            my_df = pd.read_pickle(f)
            my_df.loc[:, 'model'] = data_name
            dfs.append(my_df)
            print(data_name, f)
    #         display(my_df)
    df[dataset_name] = pd.concat(dfs, axis=0)

nb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.0/nb_log_poisson_zifa_dataset_12000_0.08_0.0.p
zinb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.0/zinb_log_poisson_zifa_dataset_12000_0.08_0.0.p
nb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.001/nb_log_poisson_zifa_dataset_12000_0.08_0.001.p
zinb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.001/zinb_log_poisson_zifa_dataset_12000_0.08_0.001.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=F

nb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.01/nb_log_poisson_zifa_dataset_12000_0.08_0.01.p
zinb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.01/zinb_log_poisson_zifa_dataset_12000_0.08_0.01.p
nb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.1/nb_log_poisson_zifa_dataset_12000_0.08_0.1.p
zinb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_0.1/zinb_log_poisson_zifa_dataset_12000_0.08_0.1.p
nb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_1.0/nb_log_poisson_zifa_dataset_12000_0.08_1.0.p
zinb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_1.0/zinb_log_poisson_zifa_dataset_12000_0.08_1.0.p
nb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_10.0/nb_log_poisson_nb_dataset_12000.p
zinb ../../ppc/scripts/log_poisson_zifa_dataset_12000_0.08_10.0/zinb_log_poisson_nb_dataset_12000.p


/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
/home/oscar/anaconda3/envs/scvienv/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=F

In [7]:
from statsmodels.stats.weightstats import ttest_ind

metrics = ['ll_ll', 't_dropout_ks_stat', 't_ratio_ks_stat', 't_cv_ks_stat']

h1_hypothesis = ['greater', 'greater', 'greater', 'greater']


df_summary = {}

n_to_keep = 50

for dataset_name in dataset_names:

    df_nb = df[dataset_name].loc[df[dataset_name].model=='nb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]
    df_zinb = df[dataset_name].loc[df[dataset_name].model=='zinb', metrics].sort_values(by='ll_ll').iloc[:n_to_keep,:]
    
    df_nb_2 = df_nb.copy()
    df_nb_2['model'] = 'nb'
    df_zinb_2 = df_zinb.copy()
    df_zinb_2['model'] = 'zinb'
    df_new = pd.concat([df_nb_2, df_zinb_2])
    
    print("Shapes")
    print(df_nb.shape)
    print(df_zinb.shape)
    print(df_new.shape)
    print(df_new)

    def get_pvals(gby, other_df):
        my_df = gby[metrics]
        print(gby)
        assert my_df.shape[1] == len(metrics)
        assert other_df.shape[1] == len(metrics)
        pvals = []
        for idx, alternative in enumerate(h1_hypothesis):
            assert len(other_df.values[:, idx]) != len(h1_hypothesis), (len(other_df.values[:, idx]), len(h1_hypothesis))
            #_, pval, _ = ttest_ind(other_df.values[:, idx], my_df.values[:, idx], alternative=alternative)
            try:
                _, pval = scipy.stats.mannwhitneyu(np.sort(other_df.values[:, idx])[:n_to_keep],
                                       np.sort(my_df.values[:, idx])[:n_to_keep][::-1],
                                       use_continuity=True,
                                       alternative=alternative)
            except ValueError:
                pval = 0.5
            
            pvals.append(pval)
        return np.array(pvals)

    pvals_against_zinb = df_new.groupby('model').apply(get_pvals, other_df=df_zinb)
    pvals_against_zinb = (pvals_against_zinb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_zinb = pvals_against_zinb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    pvals_against_nb = df_new.groupby('model').apply(get_pvals, other_df=df_nb)
    pvals_against_nb = (pvals_against_nb
             .apply(lambda x: pd.Series(x))
             .T)
    pvals_against_nb = pvals_against_nb.rename(index={idx: met for (idx,met) in enumerate(metrics)})

    def get_summary(gby):
        res = {}
        res['median'] = gby.median()
        res['pvals_against_nb'] = pvals_against_nb[gby.name]
        res['pvals_against_zi'] = pvals_against_zinb[gby.name]
        return pd.DataFrame(res).T

    # df_summary = df.groupby('model')['ll_ll', 'imputation_median_imputation_score', 't_dropout_ks_stat',
    #                                  't_ratio_ks_stat', 't_cv_ks_stat'].agg(['mean', 'std']).T
    df_summary[dataset_name] = df[dataset_name].groupby('model')[metrics].apply(get_summary)


    df_summary[dataset_name] = df_summary[dataset_name].stack().unstack(1).sort_index(level=1).swaplevel()

Shapes
(50, 4)
(50, 4)
(100, 5)
         ll_ll  t_dropout_ks_stat  t_ratio_ks_stat  t_cv_ks_stat model
39  184.307632               0.80             0.64          0.74    nb
14  184.332860               0.80             0.66          0.72    nb
36  184.363336               0.80             0.64          0.86    nb
24  184.384680               0.80             0.66          0.82    nb
5   184.394101               0.80             0.64          0.94    nb
41  184.403185               0.80             0.60          0.84    nb
8   184.407724               0.80             0.66          0.82    nb
48  184.420825               0.80             0.66          0.84    nb
16  184.422888               0.80             0.66          0.82    nb
45  184.426496               0.80             0.68          0.84    nb
32  184.447152               0.80             0.62          0.82    nb
29  184.447323               0.80             0.66          0.76    nb
37  184.466321               0.80            

         ll_ll  t_dropout_ks_stat  t_ratio_ks_stat  t_cv_ks_stat model
0   168.057715               0.24             0.22          0.62  zinb
19  168.228709               0.10             0.08          0.40  zinb
49  168.231048               0.08             0.12          0.78  zinb
20  168.472273               0.20             0.10          0.72  zinb
43  168.496261               0.16             0.16          0.40  zinb
44  168.542652               0.18             0.18          0.62  zinb
10  168.550376               0.26             0.16          0.78  zinb
18  168.555783               0.32             0.22          0.80  zinb
33  168.598778               0.26             0.20          0.24  zinb
38  168.634150               0.10             0.20          0.20  zinb
9   168.724486               0.20             0.08          0.82  zinb
46  168.732181               0.34             0.32          0.80  zinb
36  168.736444               0.08             0.12          0.44  zinb
32  16

# Export to Latex

In [11]:
def boldizer(row):
    to_bold = (row.pvals_against_nb <= 5e-2) | (row.pvals_against_zi <= 5e-2)
#     print(to_bold)
    if to_bold:
        row.latex = '\mathbf{' + row.latex + '}'
    return row

In [13]:
n_dec = 3

work_df = {}

for dataset_name in dataset_names:

    work_df[dataset_name] = (df_summary[dataset_name]
               .reset_index(level=-1)
               .reset_index()
               .assign(latex=lambda x: x['median'].round(n_dec).astype(str))
               .apply(boldizer, axis=1)
               .assign(Metric=lambda x: 
                       x.loc[:, 'index'].replace({
                           'll_ll': 'Negative LL',
                           't_cv_ks_stat': 'Coefficient of variation',
                           't_dropout_ks_stat': 'Dropout ratio',
                           't_ratio_ks_stat': 'Zeros-to-expression ratio',
                           'model': 'Model'
               }),
                       latex=lambda x: '$' + x.latex + '$'
                      )
              )

    work_df[dataset_name] = work_df[dataset_name].rename(columns={'model': 'Model', 'index': 'metric',
                                                                 'Metric': 'Metric (medians)'})
    work_df[dataset_name]['Model'] = work_df[dataset_name]['Model'].str.upper()
    #work_df[dataset_name]['Dataset'] = dataset_names[dataset_name]
    

In [17]:
pivot = {}
for dataset_name in dataset_names:
    pivot[dataset_name] = pd.pivot(work_df[dataset_name], index='Metric (medians)', values='latex', columns='Model')
    pivot[dataset_name]['Dataset'] = dataset_names[dataset_name]
    pivot[dataset_name].reset_index(inplace=True)
    pivot[dataset_name].set_index(['Dataset', 'Metric (medians)'], inplace=True)
pivot = pd.concat(list(pivot.values()), axis=0)

In [18]:
pivot.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 24 entries, ($\lambda = 0$, Coefficient of variation) to ($\lambda = 10$, Zeros-to-expression ratio)
Data columns (total 2 columns):
NB      24 non-null object
ZINB    24 non-null object
dtypes: object(2)
memory usage: 633.0+ bytes


In [19]:
res = pivot.to_latex(escape=False)
print(res)

\begin{tabular}{llll}
\toprule
               & Model &                  NB &                ZINB \\
Dataset & Metric (medians) &                     &                     \\
\midrule
$\lambda = 0$ & Coefficient of variation &              $0.84$ &     $\mathbf{0.81}$ \\
               & Dropout ratio &               $0.8$ &     $\mathbf{0.24}$ \\
               & Negative LL &           $184.885$ &  $\mathbf{170.947}$ \\
               & Zeros-to-expression ratio &              $0.66$ &     $\mathbf{0.32}$ \\
$\lambda = 0.001$ & Coefficient of variation &              $0.76$ &              $0.73$ \\
               & Dropout ratio &               $0.7$ &     $\mathbf{0.31}$ \\
               & Negative LL &           $176.854$ &   $\mathbf{170.66}$ \\
               & Zeros-to-expression ratio &               $0.6$ &     $\mathbf{0.27}$ \\
$\lambda = 0.01$ & Coefficient of variation &              $0.76$ &              $0.71$ \\
               & Dropout ratio &              $0.56$ &   